In [9]:
import os
import csv
import numpy as np
import pandas
import sys
from datetime import datetime

# main_dir = 'C:\\Users\\gkhadge\\Documents\\UCLA\\EE232E\\HW_4'
# os.chdir(main_dir)

In [12]:
#data_directory = "/Users/Yusi/Documents/EE232E/HW_4/finance_data/data"
main_dir = '/Users/Yusi/Documents/EE232E/HW_4/'

os.chdir(main_dir)
data_directory = "finance_data/data"
os.chdir(data_directory)

close_data = {}
end_date_data = {}
date_data = {}

for root,dirs,files in os.walk(os.path.join(main_dir, data_directory)):
    for file in files:
        if file.endswith(".csv"):
            with open(file) as f:
                #cf = csv.DictReader(f, fieldnames=["Close"])
                cf = csv.reader(f)
                stock_name = file[:-4]
                #cf.next()
                data = pandas.read_csv(file)
                close_data[stock_name] = data["Close"].values
                end_date_data[stock_name] = [data["Date"].values[0], data["Date"].values[-1]]
                
                # Parse into datetime format
                if stock_name[-2:] == '.B':
                    date_format = '%x'
                else:
                    date_format = '%Y-%m-%d'
                dates = [datetime.strptime(data["Date"].values[i], date_format) for i in range(len(data["Date"].values))]
                
                date_data[stock_name] = dates
                

            f.close()
            

In [ ]:
# stock_names = close_data.keys();

# for i in range(len(stock_names)):
#     if end_date_data[stock_names[i]][0] != '2014-05-01':
#         print(stock_names[i])
# #     if end_date_data[stock_names[i]][1] != '2017-05-12':
# #         print(stock_names[i])
        

In [ ]:
# r_i = np.diff(np.log(close_data['A']))
# r_j = np.diff(np.log(close_data['AAL']))

# # p_ij_n = (np.dot(r_i,r_j)/len(r_i) - np.mean(r_i)*np.mean(r_j))
# # p_ij_d = np.sqrt(np.var(r_i) * np.var(r_j))
# #p_ij_d = (np.sqrt((np.mean(np.square(r_i))-np.square(np.mean(r_i)))*(np.mean(np.square(r_j))-np.square(np.mean(r_j))))

# p_ij = (np.dot(r_i,r_j)/len(r_i) - np.mean(r_i)*np.mean(r_j))/ (np.sqrt(np.var(r_i) * np.var(r_j)))
# print(p_ij)


In [ ]:
# Correlation Graph for Daily Data
os.chdir(main_dir)
StockNetworkFile = "StockNetworkFile.txt"

f = open(StockNetworkFile, 'w')


stock_names = close_data.keys();
num_stocks = len(stock_names)

d_ij_hist = np.zeros(int(num_stocks*(num_stocks-1)/2.0), dtype=np.float)
p_ij = np.zeros(int(num_stocks*(num_stocks-1)/2.0), dtype=np.float)

ind = 0

for i in range(num_stocks):
    # Loop over all stocks
    stock_i = stock_names[i]
    for j in range(i+1,num_stocks):
        
        stock_j = stock_names[j]
        
        r_i = np.diff(np.log(close_data[stock_i]))
        r_j = np.diff(np.log(close_data[stock_j]))
        
        if len(r_i) != len(r_j):
            min_len = min(len(r_i),len(r_j))     # calculate the minimum length of r_i and r_k

            r_i = r_i[-min_len:] # since all stocks end at the same date, delete non-corresponding time values from the longer data set
            r_j = r_j[-min_len:]

#         print(len(r_i))
#         print(len(r_j))
#         print(stock_j)
        p_ij[ind] = (np.dot(r_i,r_j)/len(r_i) - np.mean(r_i)*np.mean(r_j))/ (np.sqrt(np.var(r_i) * np.var(r_j)))
        
        d_ij = np.sqrt(2*(1-p_ij[ind]))
        d_ij_hist[ind] = d_ij
        
        ind = ind+1
        
        f.write(stock_i+"\t"+stock_j+"\t"+("%.15f" % d_ij)+"\n")

f.close()


In [ ]:
date_data['ZTS'][7].weekday()


In [ ]:
# Correlation Graph for Weekly Data
os.chdir(main_dir)
StockNetworkFile = "StockNetworkFile_Weekly.txt"

f = open(StockNetworkFile, 'w')

stock_names = close_data.keys();
num_stocks = len(stock_names)

d_ij_hist = np.zeros(int(num_stocks*(num_stocks-1)/2.0), dtype=np.float)
p_ij = np.zeros(int(num_stocks*(num_stocks-1)/2.0), dtype=np.float)

ind = 0

for i in range(num_stocks):
    # Loop over all stocks
    stock_i = stock_names[i]
    for j in range(i+1,num_stocks):
        
        stock_j = stock_names[j]
        
        start_day_i = 0;
        start_day_j = 0;
        while date_data[stock_i][start_day_i] < date_data[stock_j][start_day_j]:
            start_day_i += 1
        
        while date_data[stock_i][start_day_i] > date_data[stock_j][start_day_j]:
            start_day_j += 1
            
        weekly_log_close_i = []
        weekly_log_close_j = []
        
        day = 0
        
        curr_day = -1
        while start_day_i + day < len(date_data[stock_i]) and start_day_j + day < len(date_data[stock_j]):
            prev_day = curr_day
            curr_day = date_data[stock_i][start_day_i + day].weekday()
            
            # If Monday, or Tuesday with Monday being a holiday
            if curr_day == 0 or (curr_day == 1 and prev_day == 4):
                weekly_log_close_i.append(np.log(close_data[stock_i][start_day_i + day]))
                weekly_log_close_j.append(np.log(close_data[stock_j][start_day_j + day]))
            
            day += 1
        
        r_i = np.diff(weekly_log_close_i)
        r_j = np.diff(weekly_log_close_j)
        

#         print(len(r_i))
#         print(len(r_j))
#         print(stock_j)
        p_ij[ind] = (np.dot(r_i,r_j)/len(r_i) - np.mean(r_i)*np.mean(r_j))/ (np.sqrt(np.var(r_i) * np.var(r_j)))
        
        d_ij = np.sqrt(2*(1-p_ij[ind]))
        d_ij_hist[ind] = d_ij
        
        ind = ind+1
        
        f.write(stock_i+"\t"+stock_j+"\t"+("%.15f" % d_ij)+"\n")

f.close()


In [ ]:
import matplotlib.pyplot as plt

# the histogram of the data
n, bins, patches = plt.hist(d_ij_hist, 100, normed=1, alpha = 0.75)

plt.xlabel('D_ij Value')
plt.ylabel('Probability')
plt.title('D_ij Histogram')

plt.show()

In [ ]:
# Problem 7: Modifying Correlation
os.chdir(main_dir)
StockNetworkFileMod = "StockNetworkFileMod.txt"

f = open(StockNetworkFileMod, 'w')

# open new file to save d_ij
d_ij_hist_mod = np.zeros(int(num_stocks*(num_stocks-1)/2.0), dtype=np.float)
p_ij_mod = np.zeros(int(num_stocks*(num_stocks-1)/2.0), dtype=np.float)

ind = 0

for i in range(num_stocks):
    # Loop over all stocks
    stock_i = stock_names[i]
    for j in range(i+1,num_stocks):
        
        stock_j = stock_names[j]
        
        r_i = np.diff(np.log(close_data[stock_i]))
        r_j = np.diff(np.log(close_data[stock_j]))
        
        if len(r_i) != len(r_j):
            min_len = min(len(r_i),len(r_j))     # calculate the minimum length of r_i and r_k

            r_i = r_i[-min_len:] # since all stocks end at the same date, delete non-corresponding time values from the longer data set
            r_j = r_j[-min_len:]

#         print(len(r_i))
#         print(len(r_j))
#         print(stock_j)
        p_ij_mod[ind] = (np.dot(r_i,r_j)/len(r_i) - np.mean(r_i)*np.mean(r_j))/ (np.sqrt(np.var(r_i) * np.var(r_j)))
        
        if p_ij_mod[ind] > 0.3:
            p_ij_mod[ind] = -1
        
        d_ij_mod = np.sqrt(2*(1-p_ij_mod[ind]))
        d_ij_hist_mod[ind] = d_ij_mod
        
        ind = ind + 1
        
        f.write(stock_i+"\t"+stock_j+"\t"+("%.15f" % d_ij_mod)+"\n")

f.close()

In [ ]:
# the histogram of the data
n, bins, patches = plt.hist(d_ij_hist_mod, 100, normed=1, alpha = 0.75)

plt.xlabel('D_ij Modified Value')
plt.ylabel('Probability')
plt.title('D_ij Modified Histogram')

plt.show()

In [14]:
# Eulerian Path Function
from collections import defaultdict

def find_euler_tour(graph):
    tour = []
    E = graph

    numEdges = defaultdict(int)

    def find_tour(u):
        for e in E:
            if u == e[0]:
                u,v = e
                E.remove(e)
                find_tour(v)
            elif u == e[1]:
                v,u = e
                E.remove(e)
                find_tour(v)
        tour.insert(0,u)

    for i,j in graph:
        numEdges[i] += 1
        numEdges[j] += 1

    start = graph[0][0]
    for i,j in numEdges.iteritems():
        if j % 2 > 0:
            start = i
            break

    current = start
    find_tour(current)

    if tour[0] != tour[-1]:
        return None
    return tour


In [15]:
# PROBLEM 5 TSP Approximation using Eulerian Path Method
os.chdir(main_dir)

sys.setrecursionlimit(1500)

# Load MST
with open('StockNetwork_MST.txt', 'rb') as f:
    reader = csv.reader(f, delimiter=' ')
    g_mst = list(reader)
    
# Sum MST weight, and remove edgeweights for find_euler_tour
MST_weight = 0
for i in range(len(g_mst)):
    MST_weight += float(g_mst[i][2])
    g_mst[i] = g_mst[i][0:2]
    
# Double the edges
g_mst_doubled = g_mst + g_mst

# Find Eulerian tour
etour = find_euler_tour(g_mst_doubled)

# Remove duplicate nodes to create TSP approximation
TSP_approx = [];
for i in etour:
  if i not in TSP_approx:
    TSP_approx.append(i)
# Make the TSP end at the starting node
TSP_approx.append(etour[0])

# Open the full graph
with open('StockNetworkFile.txt', 'rb') as f:
    reader = csv.reader(f, delimiter='\t')
    g = list(reader)

# Create dictionary of edge weights. keys are node1_node2 where node1 is alphabetically earlier
edgedict = {}
for edge in g:
    if edge[0] < edge[1]:
        key = edge[0]+'_'+edge[1]
    else:
        key = edge[1]+'_'+edge[0]
    edgedict[key] = float(edge[2])
    
# Sum edgeweights along TSP approx route
TSP_approx_value = 0
for i in range(len(TSP_approx)-1):
    node0 = TSP_approx[i]
    node1 = TSP_approx[i+1]
    if node0 < node1:
        edge = node0+'_'+node1
    else:
        edge = node1+'_'+node0
    TSP_approx_value += edgedict[edge]    

print "MST Weight: {:0.2f}".format(MST_weight)
print "TSP Approx: {:0.2f}".format(TSP_approx_value)

MST Weight: 426.37
TSP Approx: 485.38


In [26]:
# Triangle Inequality Check

stock_names = close_data.keys();
num_stocks = len(stock_names)

violation_count = 0
itr = 0

for i in range(num_stocks):   # iterate through all possible starting nodes
    itr = itr+1
    stock_i = stock_names[i]
    print(itr)
    for j in range(i+1,num_stocks):    # iterate through all ending nodes
        stock_j = stock_names[j]
        if stock_j != stock_i:      # make sure that the stocks are distinct
            if stock_i < stock_j
                key_ij = stock_i+'_'+stock_j     # keys are alphabetical
            else:
                key_ij = stock_j+'_'+stock_i
            l_ij = edgedict[key_ij]
            for k in range(num_stocks):      # iterate through all intermediate nodes
                stock_k = stock_names[k]
                
                if (stock_k != stock_i) and (stock_k != stock_j):      # make sure that the stocks are distinct
                    if stock_k < stock_j:
                        key_jk = stock_k+'_'+stock_j
                    else:
                        key_jk = stock_j+'_'+stock_k

                    if stock_k < stock_i:
                        key_ik = stock_k+'_'+stock_i
                    else:
                        key_ik = stock_i+'_'+stock_k
                        
                    l_ik = edgedict[key_ik]
                    l_jk = edgedict[key_jk]
                    
    if (l_jk + l_ik < l_ij):
        violation_count = violation_count+1
        
print(violation_count)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
